# Set up Pycap LPR to operate with PEST++ for optimization

In [ ]:
import yaml
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import pyemu
import matplotlib.pyplot as plt
import os, platform, shutil
from pycap.analysis_project import Project
from  datetime import date

In [ ]:
#### Path to excel input file. Note can use absolute or relative path.
pycap_inputs_excel = Path("../../PyCap/PyCap_USGS/pycap-dss2/Notebooks/Inputs/LPR_Prepped.xlsx")

#### PyCap Run Name is what all your outputs will have as a name. 
pycap_run_name = "LPR_Redux"

#### Base directory for runs
parent_run_path = Path("../pycap_runs")

#### depletion potential calculations directory
base_run_path = parent_run_path / "pycap_base"
pest_path = parent_run_path / "pycap_pest"
template_path = pest_path / "pycap_template"
if pest_path.exists():
    shutil.rmtree(pest_path)
template_path.mkdir(parents=True)

#### finally define the scripts directory
script_path = Path("../scripts")


# Parameterization for PEST++ 

In [ ]:
if not template_path.exists():
    template_path.mkdir(parents=True)

#### let's load up the configuration file

In [ ]:
with open(base_run_path / f"{pycap_run_name}.yml", 'r') as ifp:
        indat = yaml.safe_load(ifp)

#### and now parameterize inputs to vary in optimization, Monte Carlo, and other analyses

First we set up template (`TPL`) files that allow PEST++ to update model input values by name. We do this by reading in the input (`YML`) file and replacing numeric values with updated values being changed by the algorithm.

In [ ]:
# global T and S
T_init = indat['project_properties']['T']
S_init = indat['project_properties']['S']

indat['project_properties']['T'] = f'~{"global_T":^16s}~'
indat['project_properties']['S'] = f'~{"global_S":^16s}~'

In [ ]:
# now well-by-well apportionment and pumping rates
well_keys = [i for i in indat.keys() if i.startswith('well_')]
app_keys = [[j for j in indat[i].keys() if j.startswith('stream_apportionment')]
            for i in well_keys]
pending_wells = [i for i in well_keys if 'pending' in indat[i]['status']]
allkeys = dict(zip(well_keys, app_keys))

pars = list()
parvals = list()

# for apportionment
for k,v in allkeys.items():
    for cv in v:
        cpar = f'{k}__{cv}'
        pars.append(cpar)
        parvals.append(indat[k][cv]['apportionment'])
        indat[k][cv]['apportionment'] = f'~{cpar:^45}~'
# then again for pumping rate Q
for k,v in allkeys.items():
    cpar = f'{k}__q'
    pars.append(cpar)
    parvals.append(indat[k]['Q'])
    indat[k]['Q'] = f'~{cpar:^45}~'

In [ ]:
# save out tpl file
with open(template_path / f"{pycap_run_name}.yml.tpl", 'w') as ofp:
    ofp.write('ptf ~\n')
    documents = yaml.dump(indat, ofp, default_flow_style = False, sort_keys = False)

In [ ]:
# create DataFrame of parameters
pars_df = pd.DataFrame(index = pars, data= {'parval1':parvals})

# add initial parameters to df
pars_df = pd.concat([pars_df, pd.DataFrame(index = ['global_s','global_t'], data = {'parval1':[S_init,T_init]})])


In [ ]:
pars_df.sample(5)

### Next we need to be able to read in model ouputs to PEST++
Now we write an instruction file (`INS`) that can navigate model output and read it into PEST++

In [ ]:
# make ins file and external forward run file
# set base case depletion observations
basedeplobs = [f"{indat[k]['name']}:bdpl" for k in indat.keys() if 'stream' in k]

# get list of unique stream names used in the run
unique_rivers = list(set([i.split(':')[0] for i in basedeplobs]))

# add in the totals/sums of proposed/existing/combined depletions for each stream
basedeplobs.extend([f'{i}:{j}:bdpl' for i in unique_rivers for j in ['total_proposed','total_existing','total_combined']])

with open(template_path / 'basedeplobs.dat','w') as ofp:
    [ofp.write(i + '\n') for i in basedeplobs]

In [ ]:
basedeplobs

In [ ]:
# add time-series
# choose wells with individual time-series outputs.
allstrmobs = [f"{indat[k]['name']}" for k in indat.keys() if "stream" in k]
output_ts = allstrmobs 

# times you want to look at individual outputs, in this case just the depletions in the 5th year
times = range(365*4,365*5+1) 
ts_obs = []
for c_ts in output_ts:
    ts_obs.extend([f'{c_ts}__{i}' for i in times])
allobs = basedeplobs + ts_obs

with open(template_path / 'ts_obs.dat' , 'w') as ofp:
    [ofp.write(c_ts + '\n') for c_ts in output_ts]

### Now read in the base case observation values for depletion data

In [ ]:
base_data = pd.read_csv(base_run_path/"output" / f'{pycap_run_name}.table_report.base_stream_depletion.csv', index_col=0)
# read in the observation names and make a DataFrame to keep the results in
bdplobs = pd.read_csv(template_path/'basedeplobs.dat', header=None)
bdplobs.columns =['obsname']
bdplobs.index = bdplobs.obsname
bdplobs['obs_values'] = np.nan

# now map the actual output values to the DataFrame
for cob in bdplobs.obsname:
    riv,wel,_ = cob.split(':')
    bdplobs.loc[cob, 'obs_values'] = base_data.loc[wel][riv]

In [ ]:
bdplobs.sample(5)

### Next for time series - read in the results for the 5th year only for each well

In [ ]:
ts_data = pd.read_csv(base_run_path / "output" / f'{pycap_run_name}.table_report.all_ts.csv',index_col=0)
ts_data.columns = ts_data.columns.str.split('__').str[-1]
ts_path = template_path / 'ts_obs.dat' 
output_ts = [i.strip() for i in open(ts_path, 'r').readlines()]
ts_df = pd.DataFrame(index= ts_obs,  data = {'obsname':ts_obs, 'obs_values':np.nan})
for cob in ts_df.index:
    criv,ctime=cob.split('__')
    ts_df.loc[cob, 'obs_values'] = ts_data.loc[int(ctime)][criv]

In [ ]:
ts_df.sample(5)

### We can combine all the outputs into a single dataframe and make the instruction file we'll need to read in the results

In [ ]:
allout = pd.concat([bdplobs,ts_df])
allout['obs_values'].to_csv(template_path / 'allobs.out', sep = ' ', header=None)

with open(template_path / 'allobs.out.ins', 'w') as ofp:
    ofp.write('pif ~\n')
    [ofp.write(f'l1 w !{i}!\n') for i in allout.index]

### Now we need to make a PEST control file to orchestrate everything. Luckily, `pyemu` makes this straightforward now that we have made the `tpl` and `ins` files --- _Note you may see a warning about parsing metadata. This can be safely ignored._

In [ ]:
cwd = os.getcwd()
os.chdir(template_path)
pst = pyemu.Pst.from_io_files(*pyemu.utils.parse_dir_for_io_files('.'))
os.chdir(cwd)

In [ ]:
pars = pst.parameter_data

# let's clean up some of the data and add important values

In [ ]:
# name paramter groups according to the type of parameter
pars.loc[pars.parnme.str.contains("global"), "pargp"] = "global"
pars.loc[pars.parnme.str.endswith("q"), "pargp"] = "pumping"
pars.loc[pars.parnme.str.contains("stream"), "pargp"] = "apportionment"
# set initial values
pars.loc[pars_df.index,'parval1'] = pars_df.parval1

In [ ]:
pars

In [ ]:
del_pump = .2 #(as a fraction)
# set upper and lower bounds on pumping rates as well
pars.loc[pars.pargp=="pumping","parlbnd"] = pars.loc[pars.pargp=="pumping", "parval1"]*(1-del_pump)
pars.loc[pars.pargp=="pumping", "parubnd"] = pars.loc[pars.pargp=="pumping", "parval1"]*(1+del_pump)

In [ ]:
# set apportionment changes
# how much to change apportionment
del_apport = 0.1

# apport lower and upper bound
#pars.loc[pars.index.str.startswith('well_'),'parlbnd'] = pars.loc[pars.index.str.startswith('well_'),'parval1']-.1
pars.loc[pars.pargp=="apportionment", "parlbnd"] = pars.loc[pars.pargp=="apportionment", "parval1"] - del_apport
pars.loc[pars.pargp=="apportionment", "parubnd"] = pars.loc[pars.pargp=="apportionment", "parval1"] + del_apport

# force overall lowerbound and upperbound as defined in .xlsx inputs
pars.loc[(pars.pargp=='apportionment')&
    (pars.parlbnd < indat['project_properties']['Min_FracInt']),
    'parlbnd'] = indat['project_properties']['Min_FracInt']
pars.loc[(pars.pargp=='apportionment')&
    (pars.parubnd > indat['project_properties']['Max_FracInt']),
    'parubnd'] = indat['project_properties']['Max_FracInt']

In [ ]:
# set lower and upper bounds for T and S as defined in .xlsx inputs
pars.loc[(pars.parnme=='global_t'),'parlbnd'] = indat['project_properties']['Min_T']
pars.loc[(pars.parnme=='global_t'),'parubnd'] = indat['project_properties']['Max_T']

pars.loc[(pars.parnme=='global_s'),'parlbnd'] = indat['project_properties']['Min_S']
pars.loc[(pars.parnme=='global_s'),'parubnd'] = indat['project_properties']['Max_S']

# set transforms for all parameters to 'none', except for T which can be a log-transform
pars.partrans = 'none'
pars.loc['global_t','partrans']='log'

In [ ]:
pars

In [ ]:
# setup and save pst file
pst.control_data.noptmax = -1
pst.model_command = [f'python run_pycap_standalone.py']
pst.pestpp_options['par_sigma_range']=6
pst.pestpp_options['ies_num_reals'] = 50

pst.write(str(template_path / 'prior_mc.pst'), version=2)

